In [5]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models  import Sequential, load_model
from tensorflow.keras.layers import  Dense, Dropout, LSTM, Activation
from tensorflow.keras.optimizers import RMSprop


In [6]:
text_df = pd.read_csv("fake_and_real_news_dataset.csv")
text_df

,idd,title,text,label
0,Fq+C96tcx+,‘A target on Roe v. Wade ’: Oklahoma bill maki...,UPDATE: Gov. Fallin vetoed the bill on Friday....,REAL
1,bHUqK!pgmv,Study: women had to drive 4 times farther afte...,Ever since Texas laws closed about half of the...,REAL
2,4Y4Ubf%aTi,"Trump, Clinton clash in dueling DC speeches","Donald Trump and Hillary Clinton, now at the s...",REAL
3,_CoY89SJ@K,Grand jury in Texas indicts activists behind P...,A Houston grand jury investigating criminal al...,REAL
4,+rJHoRQVLe,"As Reproductive Rights Hang In The Balance, De...",WASHINGTON -- Forty-three years after the Supr...,REAL
...,...,...,...,...
4589,ukZm6JTO#x,Russia Calls the War Party's Bluff,License DMCA \nCold War 2.0 has reached unprec...,FAKE
4590,yu0xKEiapJ,Bernie Sanders: The Democratic primary gave me...,Print \nSen. Bernie Sanders laid out the ways ...,FAKE
4591,c4Y370E_9c,"Pipeline Police Strip Search Native Girl, Then...",As the pressure to start construction on the D...,FAKE
4592,bBbeuCUeMH,Currency Crisis: Alasdair MacLeod On The Vexed...,Tweet Home » Gold » Gold News » Currency Crisi...,FAKE


In [7]:
text = list(text_df.text.values)
joined_text = "".join(text)

In [8]:
partial_text = joined_text[:100000]

In [9]:
tokenizer = RegexpTokenizer(r"\w+") # Just Words
tokens = tokenizer.tokenize(partial_text.lower())

In [10]:
unique_tokens = np.unique(tokens)
unique_token_index = {token: idx for idx, token  in enumerate(unique_tokens)} #indexed Dictionary of tokens 

In [11]:
# Categorize Input Words and Next Words 
n_words = 5 # How far Ahead
input_words = []
next_words = []

for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])

In [12]:

X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [13]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
        y[i,unique_token_index[next_words[i]]] = 1

In [14]:
model = Sequential()
model.add(LSTM(128, input_shape = (n_words, len(unique_tokens)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens), activation='softmax'))

In [15]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.001), metrics=["accuracy"])
model.fit(X, y, batch_size=128, epochs=500, shuffle=True)

Epoch 1/1000




In [ ]:
model.save("test_model.h5")

test_model =load_model('test_model.h5')

NameError: name 'model' is not defined

In [ ]:
## prediction
def predict_next_word(input_text, n_best):
    input_text = input_text.lower()
    X = np.zeros((1, n_words, len(unique_tokens)))
    for t, word in enumerate(input_text.split()):
        X[0, t, unique_token_index[word]] = 1

    predictions = model.predict(X)[0]
    top_n = np.argpartition(predictions, n_best)[n_best:]
    return [unique_tokens[i] for i in top_n]



In [ ]:
possible = predict_next_word("He will have to take into this", 4)
print(possible)

IndexError: index 5 is out of bounds for axis 1 with size 5